In [15]:
from itertools import combinations
import pprint

In [7]:
def get_itemsets(transactions, itemset_size): # form cominations of size k
    itemsets = set()
    for transaction in transactions:
        for combination in combinations(transaction, itemset_size):
            itemsets.add(frozenset(combination))
    return itemsets

In [8]:
def get_frequent_itemsets(transactions, itemsets, min_support): # n square
    itemset_counts = {itemset: 0 for itemset in itemsets} 
    for transaction in transactions:
        for itemset in itemsets:
            if itemset.issubset(transaction):
                itemset_counts[itemset] += 1

    frequent_itemsets = {itemset: count for itemset, count in itemset_counts.items() if count >= min_support}
    return frequent_itemsets, itemset_counts

In [10]:
def print_table(itemset_counts, min_support):
    for item, count in itemset_counts.items():
        print(f" {str(set(item))}   {count}  {'✓' if count >= min_support else '✗'}")

    print()

In [12]:
def generate_new_combinations(frequent_itemsets, k): # new itemsets
    itemsets = set()
    frequent_items = list(frequent_itemsets)

    for i in range(len(frequent_items)):
        for j in range(i + 1, len(frequent_items)):
            union_set = frequent_items[i] | frequent_items[j]
            if len(union_set) == k: 
                itemsets.add(union_set)

    return itemsets

In [ ]:
def apriori(transactions,min_support):
    transactions = [frozenset(t) for t in transactions]
    itemsets = get_itemsets(transactions,1)
    frequent_itemsets, itemset_counts = get_frequent_itemsets(transactions, itemsets, min_support)

    print(f"Step 1: Frequent 1-itemsets")
    print_table(itemset_counts, min_support)
    
    all_frequent_itemsets = dict(frequent_itemsets)
    
    k = 2
    while frequent_itemsets:
        itemsets = generate_new_combinations(frequent_itemsets.keys(), k)
        frequent_itemsets, itemset_counts = get_frequent_itemsets(transactions, itemsets, min_support)

        if frequent_itemsets:
            print(f"Step {k}: Frequent {k}-itemsets")
            print_table(itemset_counts, min_support)

        all_frequent_itemsets.update(frequent_itemsets)
        k += 1

    return all_frequent_itemsets
    

In [16]:
# Example
transactions = [
    ['milk', 'bread', 'butter'],
    ['bread', 'butter'],
    ['milk', 'bread'],
    ['milk', 'butter'],
    ['bread', 'butter'],
]

min_support = 2
frequent_itemsets = apriori(transactions, min_support)
print("Final Frequent Itemsets:")
pprint.pprint(frequent_itemsets)

Step 1: Frequent 1-itemsets
 {'butter'}   4  ✓
 {'milk'}   3  ✓
 {'bread'}   4  ✓

Step 2: Frequent 2-itemsets
 {'butter', 'bread'}   3  ✓
 {'milk', 'butter'}   2  ✓
 {'milk', 'bread'}   2  ✓

Final Frequent Itemsets:
{frozenset({'butter'}): 4,
 frozenset({'milk'}): 3,
 frozenset({'bread'}): 4,
 frozenset({'butter', 'bread'}): 3,
 frozenset({'milk', 'butter'}): 2,
 frozenset({'milk', 'bread'}): 2}


#### Get association rules

In [25]:
# LHS -> RHS
def get_ar(frequent_itemsets,min_confidence):
    rules = []
    for item, count in frequent_itemsets.items():
        if len(item) > 1: 
            for size in range(1,len(item)):
                for rhs in combinations(item,size):
                    lhs = item - frozenset(rhs)
                    if lhs:
                        lhs_support = frequent_itemsets[lhs]
                        confidence = count / lhs_support

                        if confidence >=min_confidence: 
                            rules.append((lhs, frozenset(rhs), confidence))
                    

    return rules

In [27]:
def print_ar(rules):
    for lhs, rhs, confidence in rules:
        print(f"{str(set(lhs))} -> {str(set(rhs)):<20} {confidence:<10.2f}")
    print()

In [28]:
min_confidence = 0.6
association_rules = get_ar(frequent_itemsets, min_confidence)
print_ar(association_rules)

{'bread'} -> {'butter'}           0.75      
{'butter'} -> {'bread'}            0.75      
{'milk'} -> {'butter'}           0.67      
{'milk'} -> {'bread'}            0.67      

